In [1]:
from utils.utils import *

In [61]:
import torch
from accelerate import Accelerator
from accelerate import PartialState
from accelerate.utils import gather_object
import argparse
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, TextStreamer, BitsAndBytesConfig
from utils.utils import *
from utils.metrics import *
import pickle
import argparse

def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model_path", type=str)
    parser.add_argument("--test_data", type=str)
    parser.add_argument("--output_dir", type=str)
    parser.add_argument("--max_new_tokens", type=int)
    parser.add_argument("--batch_size", type=int, default=1)
    parser.add_argument('--use_flash_attention_2', action='store_true')
    parser.add_argument('--add_bos_token', action='store_true')
    args,_ = parser.parse_known_args()
    return args


def get_tokenizer_and_model(args):
    tokenizer = AutoTokenizer.from_pretrained(args.model_path, trust_remote_code=True)
    if args.use_flash_attention_2:
        model = AutoModelForCausalLM.from_pretrained(args.model_path, device_map={"": accelerator.process_index}, attn_implementation="flash_attention_2", trust_remote_code=True, torch_dtype=torch.bfloat16)
    else:
        model = AutoModelForCausalLM.from_pretrained(args.model_path, trust_remote_code=True)
    tokenizer.padding_side='left'
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    return tokenizer, model

# QA
def get_prompt(tokenizer, title, context, question, tokenize=False, add_bos_token=False):
    
    document = title+' '+context
    prompt = f"질문:{question}\n지문:{document}\n답변:"
    if add_bos_token:
        prompt = tokenizer.bos_token + prompt
    return prompt
# def get_prompt(tokenizer, premise, hypothesis, tokenize=False, add_bos_token=False):
#     prompt = f"전제: {premise}\n가설: {hypothesis}\n주어진 가설과 전제는 어떠한 관계인가\n{OPTIONS}"#prompt = f"질문:{question}\n지문:{document}\n답변:"
#     if add_bos_token:
#         prompt = tokenizer.bos_token + prompt
#     return prompt

In [53]:
from utils.data_utils import *

In [2]:
seed_everything(42)
args = get_args()
accelerator = Accelerator()

In [62]:
! ls ../kt_output

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


data			     midm_10b_plm_70k
domain_refine_flatten	     midm_v3_normal_adjusted
domain_refine_merge_version  midm_v3_only_josa_adjusted
gpt4			     nli
heuristic_filtering	     remove_duplicate
llama3.1


In [87]:
args.model_path = f'../kt_output/midm_v3_normal_adjusted/qa/model_2'
args.test_data = '/home/work/g-earth-22/hoyoun/downstream/data/KorQuad_dev.jsonl'
# args.test_data = '/home/work/user/ocw/SFT/data/klue_nli_dev.jsonl'
args.use_flash_attention_2=True
args.max_new_tokens=64
args.add_bos_token = False

In [88]:
# tokenizer, model
tokenizer, model  = get_tokenizer_and_model(args)
test_data = load_jsonl(args.test_data)
batch_size = args.batch_size

5774it [00:00, 87395.21it/s]


In [65]:
# normal

In [89]:
for i in test_data:
    prompt = get_prompt(tokenizer, i['title'],i['context'], i['question'], tokenize=False, add_bos_token=args.add_bos_token)
#     prompt = get_prompt(tokenizer, i['premise'], i['hypothesis'], tokenize=False, add_bos_token=args.add_bos_token)
    i['input']=prompt

terminators = [tokenizer.eos_token_id]
streamer = None
if accelerator.is_main_process and batch_size == 1:
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True) 
completions_per_process = []
n_samples = 50
for d in tqdm(test_data[:n_samples], disable=accelerator.is_main_process != True):
    batch = tokenizer(d['input'], add_special_tokens=False, return_tensors='pt')
    batch = batch.to(accelerator.device)
    length = batch['input_ids'].size(1)
    print(length)
    # just greedy
    outputs = model.generate(input_ids = batch['input_ids'], streamer=streamer, eos_token_id=terminators, pad_token_id = tokenizer.eos_token_id, num_beams=1, max_new_tokens=args.max_new_tokens, do_sample=False)
    outputs = outputs[:,length:].contiguous()
    generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    completions_per_process.extend(generated_text)
#     print(LABEL2STR[d['label']])
#     print('=========================================')
#     input()        


  0% 0/50 [00:00<?, ?it/s]

250
1989년 2월 15일


  2% 1/50 [00:00<00:15,  3.09it/s]

253
임수경


  4% 2/50 [00:00<00:09,  4.81it/s]

...

 94% 47/50 [00:06<00:00,  7.60it/s]

313
8층
310
고페르나무


 98% 49/50 [00:07<00:00,  8.24it/s]

161
기독교


100% 50/50 [00:07<00:00,  7.06it/s]


In [90]:
normal = copy.deepcopy(completions_per_process)

In [85]:
josa = copy.deepcopy(completions_per_process)

In [7]:
from utils.metrics import *

In [86]:
get_scores(normal,[[i['answer']] for i in test_data[:n_samples]])

{'f1': 0.846952380952381, 'em': 0.76}

In [91]:
get_scores(josa,[[i['answer']] for i in test_data[:n_samples]])

{'f1': 0.846952380952381, 'em': 0.76}

In [92]:
normal = load_jsonl('/home/work/user/ocw/kt_output/midm_v3_normal_adjusted/qa/model_2/attached.jsonl')

5774it [00:00, 47683.38it/s]


In [93]:
josa = load_jsonl('/home/work/user/ocw/kt_output/midm_v3_only_josa_adjusted/qa/model_2/attached.jsonl')

5774it [00:00, 50908.23it/s]


In [96]:
get_scores([i['predict'] for i in josa],[[i['answer']] for i in normal])

{'f1': 0.6395260312561997, 'em': 0.584170419120194}

In [97]:
get_scores([i['predict'] for i in normal],[[i['answer']] for i in normal])

{'f1': 0.5314202692969565, 'em': 0.4823346033945272}

In [135]:
differences = []
# both_wrongs = []
just_normal_wrongs = []
for i,j in zip(normal, josa):
    if i['answer']!=i['predict'] and j['answer']==j['predict']:
        
#     if i['predict']!=j['predict']:
        normal_predict = i['predict']
        josa_predict = j['predict']
        tmp = copy.deepcopy(i)
        tmp['normal_predict']=normal_predict
        tmp['josa_predict']=josa_predict
#         differences.append(copy.deepcopy(tmp))
#         both_wrongs.append(tmp)
        just_normal_wrongs.append(tmp)

In [111]:
normal_wrong = []
josa_wrong = []
for i in differences:
#     print('document')
#     print(i['title']+' '+i['context'])
#     print('question')
#     print(i['question'])
    if i['normal_predict']!=i['answer']:
        normal_wrong.append(i)
        # normal이 틀린 경우
#         print('normal_wrong')
#         print(f'normal ----- %s'%i['normal_predict'])
#         print(f'josa ----- %s'%i['josa_predict'])
#         print(f'answer ----- %s'%i['answer'])
    elif i['josa_predict']!=i['answer']:
        josa_wrong.append(i)
#         print('normal_wrong')
#         print(f'normal ----- %s'%i['normal_predict'])
#         print(f'josa ----- %s'%i['josa_predict'])
#         print(f'answer ----- %s'%i['answer'])
#     input()

In [136]:
samples = random.sample(just_normal_wrongs, k=20)

In [137]:
# 


In [138]:
for i in samples:
    print('document')
    print(i['title']+' '+i['context'])
    print('question')
    print(i['question'])
    print('normal_wrong')
    print(f'normal ----- %s'%i['normal_predict'])
    print(f'josa ----- %s'%i['josa_predict'])
    print(f'answer ----- %s'%i['answer'])
#     elif i['josa_predict']!=i['answer']:
#         josa_wrong.append(i)
# #         print('normal_wrong')
# #         print(f'normal ----- %s'%i['normal_predict'])
# #         print(f'josa ----- %s'%i['josa_predict'])
# #         print(f'answer ----- %s'%i['answer'])
    i['memo']=input()

document
우루과이_축구_국가대표팀 FIFA에서는 월드컵 우승 팀에게 유니폼에 '우승'을 상징하는 '별'을 달 것을 권장하고 있다. 우루과이는 2회 월드컵 우승(1930, 1950)에 더해, 월드컵 창립 이전에 실시된 올림픽경기대회 축구 종목에서의 금메달 2회 획득(1924, 1928)을 근거로, 총 별 4개를 유니폼에 새겨넣고 있다. 1920년대의 올림픽 우승을 기리는 이유는 당시는 월드컵 대회가 출범하기 이전이었으며, 유럽팀이 최강이라는 인식이 자리 잡고 있던 시기에 올림픽 축구를 남미국가가 제패함으로써, 아마추어 선수로 출전자격이 제한된 올림픽 무대와는 별도로 진정한 세계 최강팀을 가리는 대회를 만들자는 공감대가 형성되었으며, 그로인해 1930년 제1회 우루과이 월드컵이 개최되는 계기가 되었기 때문이다.
question
FIFA월드컵 우승팀이 유니폼에 다는 것은?
normal_wrong
normal ----- 4개
josa ----- 별
answer ----- 별


 원하는 것을 더 못찾음


document
이회창 1997년 대선 당시 한나라당 이회창 후보는 지역감정 문제에 대한 공방에 "영남출신 대의원들이 비영남인 저를 뽑아준 이 정당이 지역주의입니까? 90 몇 %를 차지하고 전혀 다른 지역 출신의 개입을 허용하지 않는 정당이 지역주의 정당입니까?"라는 발언으로 지역감정을 유발한다. 또 12월 16일 서울에서 열린 선거 유세에서는 "여러분이 김대중 후보를 당선시켜서 이 나라를 혼란에 빠뜨리고 불안정하게 하는 것이 소망이라고 생각하신다면은 그러면 이인제나 김대중 후보에게 표를 던지십시요."라는 위험수위를 넘는 발언으로 논란을 빚었다. 다음날인 12월 17일 부산에서 열린 선거 유세에서는 "이분(이인제)에게 던지는 표는 이것은 바로 아무런 효과도 내지 못하는 죽은 표가 될 뿐 아니라 바로 우리가 걱정하는 김대중 총재를 위한 표가 될 것입니다"라는 발언으로 반호남정서를 유발하는 발언을 한다.
question
1997년 당시 이회창의 발언으로 무엇을 유발하였나?
normal_wrong
normal ----- 김대중
josa ----- 지역감정
answer ----- 지역감정


 원하는 것을 못찾음


document
대한민국_아파트의_역사 1941년과 1944년에 부산토지공사가 지은 청풍장-소화장아파트는 부산광역시에 지어진 최초의 4층 짜리 아파트이다. 초기에는 아파트와 부산광역시청 겸용으로 지었으나 이후 건물 전체가 아파트로 사용되었으며 부산 최고의 공동주택이었는데 1980년대까지 외관은 아주 노후해서 놀라고, 안에 들어가면 넓어서 또 놀라고, 그리고 집주인이 부자라서 놀라서 세 번 놀란다는 말까지 생길 정도로 당시 최고급 아파트라는 것을 인증했다. 그러나 1996년 안전진단에서 D등급을 판정받았으나 부지가 작아 재개발이 시행되지 않고 있다. 그러다가 문화재청이 등록문화재로 지정하려고 부산광역시에 알리면서 관심이 커졌으며 문화유산으로 지정은 되지 않았지만 부산광역시의 근대문화유산으로 지정, 보존을 추진하고 있다.
question
청풍장-소화장아파트는 부산광역시에서 등록문화재 대신 무엇으로 지정되었는가?
normal_wrong
normal ----- 문화재
josa ----- 근대문화유산
answer ----- 근대문화유산


...

 원하는 것을 못찾음


document
바이오쇼크 게임은 1960년 랩쳐가 붕괴된 직후 잭(게임의 주인공)이 탄 비행기가 대서양을 지나는 중 불시착하는 것으로 시작한다. 추락 후에, 잭은 이 참사에서 살아남은 사람은 자신밖에 없음을 알게 되고, 가까운 곳에 등대가 설치된 섬을 발견하고 수영해 간다. 그 곳에는 수중 도시인 랩쳐로 들어가기 위한 구형 잠수기가 설치되어 있었다. 아일랜드 사람이라고 밝힌 아틀라스는 잠수함에 있는 무전기로 잭을 안전하게 인도한다. 그러는 동안 라이언은 잭을 육지 국가에서 온 요원으로 믿고 랩쳐의 자동화 시스템과 그의 페로몬으로 조정하는 스플라이서를 이용해 공격하게 한다. 아틀라스는 잭에게 살 수 있는 유일한 방법은 플라스미드로부터 받은 능력을 이용하고, ADAM을 추출하기 위해 반드시 리틀 시스터를 죽여야 한다고 말한다. 아틀라스의 말을 우연히 들은 테넌바움 박사는 잭에게 리틀 시스터를 살려야 한다고 강하게 주장하며, 각각의 리틀 시스터가 깊숙히 갖고 있던 바다 달팽이를 바꿔 플라스미드로 줄 거라고 말한다. 아틀라스는 그의 아내와 자식들이 잠수함에 숨어 있다고 하며 잭을 잠수함으로 인도한다. 잭과 아틀라스가 잠수함이 있는 만(灣)에 도착하는데 그때 라이언이 그것을 파괴시킨다. 분노한 아틀라스는 잭에게 라이언을 죽여달라고 요청한다.
question
잠수함에 있는 무전기로 잭을 안전하게 인도한 아일랜드 사람의 이름은 무엇인가?
normal_wrong
normal ----- 테넌바움
josa ----- 아틀라스
answer ----- 아틀라스


 원하는 것을 못찾음


document
대한민국_아파트의_역사 개포주공아파트는 1981년 현대건설이 지은 주공아파트이다. 또한 전두환 정권이 도입하였던 택지개발촉진법의 첫 사례이기도 하다. 1980년대 당시 강남개발로 인해 사람들이 강남에 몰리자 정부는 주택난을 해소하기 위하여 주공아파트를 여러 곳에 짓기로 결심한다. 이곳 중 잘 알려진 곳이 개포주공아파트와 개포시영아파트이다. 개포주공아파트는 1단지에서 7단지까지 지어졌으며 현재 재개발에 들어갈 예정이다. 서울시는 이전부터 재개발계획이 있었으나 여러 문제로 인해 미루어지다 최근 들어서 조합이 설립되는 등 재개발시행에 본격적으로 들어갔다. 하지만 서울시의 층수제한에 많은 논란이 일면서 갈등이 있었으나 현재는 용적률 300%로 동의가 된 상태로 이제 최종허가와 보상만 하면 되는 실정이다.
question
개포주공아파트는 몇 단지로 이루어져 있나?
normal_wrong
normal ----- 개포주공아파트
josa ----- 7단지
answer ----- 7단지


KeyboardInterrupt: Interrupted by user

In [140]:
tokenizer('개포주공아파트는 몇 단지로 이루어져 있나?')

{'input_ids': [539, 714, 31855, 7531, 271, 2039, 40676, 11852, 11964, 33], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [142]:
tokenizer_josa = AutoTokenizer.from_pretrained('../kt_output/midm_v3_only_josa_adjusted/qa/model_2')

'개'

In [147]:
tokenizer('개포주공아파트는 몇 단지로 이루어져 있나?')

{'input_ids': [539, 714, 31855, 7531, 271, 2039, 40676, 11852, 11964, 33], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [165]:
for i in tokenizer.encode('스테판 벨로프 선수가 가장 빠른 랩타임을 기록한 당시 탔던 차량은?'):
    print(tokenizer.decode([i]))

스테
판
 벨
로프
 선수가
 가장
 빠른
 랩
타임을
 기록한
 당시
 탔
던
 차량은
?


In [166]:
for i in tokenizer_josa.encode('스테판 벨로프 선수가 가장 빠른 랩타임을 기록한 당시 탔던 차량은?'):
    print(tokenizer_josa.decode([i]))
    

스테
판
 벨
로프
 선수
가
 가장
 빠른
 랩
타임
을
 기록한
 당시
 탔
던
 차량
은
?


In [123]:
from collections import defaultdict,Counter

In [124]:
Counter([i['memo'] for i in samples])

Counter({'원하는 부분을 못찾음': 6, '맞음': 3, '맞음 (분석 필요)': 1})

In [ ]:
defaultdict()

In [112]:
len(normal_wrong)

2438

In [113]:
len(josa_wrong)

596

In [102]:
differences[0][1]['predict']

'1989년 2월 15일'

In [77]:
get_scores(josa,[[i['answer']] for i in test_data[:n_samples]])

{'f1': 0.7373333333333334, 'em': 0.64}